In [1]:
import os
import torch
import random
from torchvision import transforms
from PIL import Image
import pandas as pd

In [2]:
torch.manual_seed(1337)

In [3]:
images_dir = "../data/images"
output_dir = "../data/augmented_images"
os.makedirs(output_dir, exist_ok=True)

In [4]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=random.uniform(0, 1)),
    transforms.RandomVerticalFlip(p=random.uniform(0, 1)),
    transforms.RandomRotation(degrees=random.uniform(1, 50)),
    transforms.ColorJitter(
        brightness=random.uniform(0.6, 1.4),
        contrast=random.uniform(0.6, 1.4),
        saturation=random.uniform(0.6, 1.4),
        hue=random.uniform(0.01, 0.1)
    ),
    transforms.RandomResizedCrop(
        size=(256, 256),
        scale=(0.8, 1.2),
        ratio=(0.8, 1.2)
    ),
    transforms.GaussianBlur(kernel_size=random.choice([3, 5])),
    transforms.ToTensor()
])

In [5]:
transform

Compose(
    RandomHorizontalFlip(p=0.6303561385056857)
    RandomVerticalFlip(p=0.4232643414731535)
    RandomRotation(degrees=[-36.05797970723887, 36.05797970723887], interpolation=nearest, expand=False, fill=0)
    ColorJitter(brightness=(0.35148457354133245, 1.6485154264586677), contrast=(0.3064557776705268, 1.6935442223294732), saturation=(0.11887096097337191, 1.8811290390266282), hue=(-0.01599377299636349, 0.01599377299636349))
    RandomResizedCrop(size=(256, 256), scale=(0.8, 1.2), ratio=(0.8, 1.2), interpolation=bilinear, antialias=warn)
    GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0))
    ToTensor()
)

In [6]:
num_augmented_images = 50
labeled_data = []

In [7]:
for filename in os.listdir(images_dir):
    # Load the image
    image_path = os.path.join(images_dir, filename)
    image = Image.open(image_path)
    image_tensor = transforms.ToTensor()(image)
    flattened_tensor = image_tensor.view(-1)
    labeled_data.append((flattened_tensor, 1))


    for i in range(num_augmented_images):
        # Apply the transformations
        augmented_image = transform(image)
        labeled_data.append((augmented_image.view(-1), 1))
        # Save the augmented image
        # output_filename = f"{os.path.splitext(filename)[0]}_augmented_{i}.png"
        # output_path = os.path.join(output_dir, output_filename)
        # transforms.ToPILImage()(augmented_image).save(output_path)

In [8]:
for item in labeled_data:
    assert(item[0].dtype == torch.float32)

In [10]:
labeled_data

[(tensor([0.1098, 0.1098, 0.1098,  ..., 1.0000, 1.0000, 1.0000]), 1),
 (tensor([0.0549, 0.0549, 0.0706,  ..., 1.0000, 1.0000, 1.0000]), 1),
 (tensor([0.3569, 0.3569, 0.3686,  ..., 1.0000, 1.0000, 1.0000]), 1),
 (tensor([0.0039, 0.0039, 0.0039,  ..., 1.0000, 1.0000, 1.0000]), 1),
 (tensor([0.0314, 0.0314, 0.0314,  ..., 1.0000, 1.0000, 1.0000]), 1),
 (tensor([0., 0., 0.,  ..., 1., 1., 1.]), 1),
 (tensor([0.0235, 0.0235, 0.0235,  ..., 1.0000, 1.0000, 1.0000]), 1),
 (tensor([0.2471, 0.2471, 0.2471,  ..., 1.0000, 1.0000, 1.0000]), 1),
 (tensor([0.0000, 0.0039, 0.0078,  ..., 1.0000, 1.0000, 1.0000]), 1),
 (tensor([0.2745, 0.2745, 0.2745,  ..., 1.0000, 1.0000, 1.0000]), 1),
 (tensor([0.0118, 0.0118, 0.0118,  ..., 1.0000, 1.0000, 1.0000]), 1),
 (tensor([0.0941, 0.0941, 0.0941,  ..., 1.0000, 1.0000, 1.0000]), 1),
 (tensor([0., 0., 0.,  ..., 1., 1., 1.]), 1),
 (tensor([0.1294, 0.1294, 0.1294,  ..., 1.0000, 1.0000, 1.0000]), 1),
 (tensor([0.0353, 0.0353, 0.0353,  ..., 1.0000, 1.0000, 1.0000]), 1)

In [11]:
image_data, labels = zip(*labeled_data)
df = pd.DataFrame({"data": image_data, "label": labels})

In [12]:
len(labeled_data)

204